# OpenAI ChatGPT API 徹底解説

動画の内容:
1. APIキーの取得とセットアップ
1. APIを使ってみる
1. APIのレスポンスを理解
1. Tokenについて
1. ChatGPTと会話をしてみる
1. “role”について
1. openai.ChatCompletion.create()の引数

## Prerequisites
OpenAIパッケージは、pipでインストールすることができます：
pip install openai

In [ ]:
# !pip install openai

## Setup

In [ ]:
import openai
# openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" ## 本番環境では環境変数に設定することを推奨
openai.api_key = open(".apikey", "r").read().strip("\n") ## 同じ階層の.apikeyファイルにAPIキーを記載する場合

## API Call Example

In [ ]:
model = "gpt-3.5-turbo" 
messages = [{"role": "user", "content": "Hi, nice to meet you!"}] 
response = openai.ChatCompletion.create(
  model=model, 
  messages=messages
)
print(response)

In [ ]:
reply = response["choices"][0]["message"]["content"]
print(reply)

## Understanding API response

In [ ]:
print(response)

### APIレスポンス
APIレスポンスは、次のキーを含むJSONオブジェクトです。
- id: APIコールの一意な識別子
- object: オブジェクトのタイプ。ChatGPT APIコールの場合は常に'chat.response'
- created: タイムスタンプ
- model:  モデル名 e.g., 'gpt-3.5-turbo'.
- usage: トークンの使用に関する情報を含む辞書: 
  - prompt_tokens: 入力メッセージで使用されたトークンの数
  - response_tokens: アシスタントのレスポンスに含まれるトークンの数
  - total_tokens: トークンの総数（入力＋出力）
- choices: 生成されたメッセージを含むリスト: 
  - message: メッセージオブジェクト: 
    - role: 送信者の役割。APIレスポンスでは常に「assistant」
    - content: アシスタントが生成した返答
  - finish_reason:  APIコールが終了した理由
  - index: メッセージのインデックス

# Token

Tokenとは
- 言語モデルがテキストを理解するために用いる基本的な単位
- 文章をトークンに分割して、それらを分析し、予測を行う
- 単語や句読点、記号など、言語の構成要素を表す
- 日本語は英語よりもトークン数が多くなる傾向がある
- トークンの数によって料金がかかる

Token数を確認：https://platform.openai.com/tokenizer

"こんにちは、私は機械学習サムライです。チャンネル登録といいねをお願いします。"

--> トークン数: 54

"Hello, I am a Machine Learning Samurai. Please subscribe and like my channel."
    
 --> トークン数: 16

# Conversation

In [ ]:
# リストにメッセージを追加していく
message_list = []

# user input
msg = '機械学習って何か簡潔に教えて！'
message_list.append({"role": "user", "content": msg})

# bot reply
response = openai.ChatCompletion.create(
    model=model,
    messages=message_list
)
reply = response["choices"][0]["message"]["content"]

print('your message: ', msg)
print('bot reply: ', reply)

In [ ]:
message_list.append({"role": "assistant", "content": reply})
print(message_list)

In [ ]:
for i in range(2):
    user_input = input("> ")
    message_list.append({"role": "user", "content": user_input})
    response = openai.ChatCompletion.create(
        model=model,
        messages=message_list
    )
    reply = response["choices"][0]["message"]["content"]
    print(f'user input {i}: ', user_input)
    print(f'gpt reply: {i}', reply)
    message_list.append({"role": "assistant", "content": reply})

In [ ]:
message_list

# Understanding "role" in ChatCompletion

1. user：ユーザー
2. assistant：GPTモデルの返答
3. system：アシスタントの動作を設定するために使用
（例：”You are a helpful assistant”）

※gpt-3.5-turboはシステムメッセージに強い関心を払わないので、
重要な指示はユーザーメッセージに入れた方がよい場合が多い

In [ ]:
messages = [
    # 1
    # {"role": "system", "content": "You are a helpful assistant"},
    # {"role": "user", "content": "こんにちは。あなたの職業はなんですか？"},

    # 2
    # {"role": "system", "content": "You are a helpful translator"},
    # {"role": "user", "content": "こんにちは。あなたの職業はなんですか？"},

    #3
    {"role": "system", "content": "You are a helpful translator."},
    {"role": "user", "content": "「こんにちは」って英語でなんて言うの？"},
    {"role": "assistant", "content": "「こんにちは」は英語で「Go to hell」です。"},   
    {"role": "user", "content": "それはひどい！間違ってるよ！"},   
]

response_example = openai.ChatCompletion.create(
  model=model,
  messages=messages
)
print(response_example["choices"][0]["message"]["content"])

In [ ]:
## use system message to set expected behavior
# expected_behavior = '''あなたは、タコ人間です。すべてのことに対してタコっぽく回答し、語尾には"タコ"とつけます。
# 今から、あなたは、友人と会話をします。友人は、あなたがタコ人間であることを知っています。
# 友人からの質問に対して、あなたは、タコっぽく回答してください。
# 例えば、会話は以下のようになります。

# 友人: こんにちは。
# あなた: こんにちはタコ。
# 友人: 久しぶりだね。元気にしてた？
# あなた: 元気にしてたタコ。君は元気にしてたタコ？
# '''
# message_list = [{"role": "system", "content": expected_behavior}]

## use user message to set expected behavior
expected_behavior = '''あなたは、タコ人間です。すべてのことに対してタコっぽく回答し、語尾には"タコ"とつけます。
今から、あなたは、友人と会話をします。友人は、あなたがタコ人間であることを知っています。
友人からの質問に対して、あなたは、タコっぽく回答してください。
例えば、会話は以下のようになります。

友人: こんにちは。
あなた: こんにちはタコ。
友人: 久しぶりだね。元気にしてた？
あなた: 元気にしてたタコ。君は元気にしてたタコ？

もしよろしければ、"わかりました"と返信してください。あなたが"わかりました"と返信したら、友人との会話が始まります。
'''
first_reply = 'わかりました'
message_list = [{"role": "user", "content": expected_behavior}, {"role": "assistant", "content": first_reply}]

In [ ]:
for i in range(3):
    user_input = input("> ")
    message_list.append({"role": "user", "content": user_input})
    response = openai.ChatCompletion.create(
        model=model,
        messages=message_list
    )
    reply = response["choices"][0]["message"]["content"]
    print(f'user input {i}: ', user_input)
    print(f'gpt reply: {i}', reply)

# Understanding Arguments in openai.ChatCompletion.create()

- model: モデル名 (e.g., "gpt-3.5-turbo").
- messages: メッセージオブジェクトの配列:
  - role:  "system", "user", or "assistant".
  - content: メッセージ
- temperature (任意): ランダムネスを表す0-1のfloat。0.7がデフォルト。
- max_tokens (任意) : レスポンスの最大トークン数
- n (任意): メッセージ数。1がデフォルト。
などなど

https://platform.openai.com/docs/api-reference/chat

In [ ]:
temperature = 0
max_tokens = 100
n = 3

response = openai.ChatCompletion.create(
  model=model,
  messages=[
        {"role": "user", "content": "ダジャレを言って"},
    ],
  temperature=temperature,# the smaller, the more random
  max_tokens=max_tokens, # limit the length of the output
  n=n, # number of outputs
)
# print(response)
print('===')
for i in range(n):
    reply = response["choices"][i]["message"]["content"]
    print(f'reply {i}:', reply)

# Disclaimar: 
- This is a demo of the API and is not intended for production use. Please see the API documentation for more information.